In [10]:
import numpy as np

# Convolutional Encoder
def conv_encoder(data, g):
    state = np.zeros(len(g[0]), dtype=int)
    encoded = []
    for bit in data:
        state = np.insert(state, 0, bit)[:-1]
        encoded_bit = [(bit + np.dot(state, g_i)) % 2 for g_i in g]
        encoded.append(encoded_bit)
    return np.array(encoded).flatten()

# Interleaver
def interleave(data):
    np.random.seed(42)  # Seed for reproducibility
    interleaver_sequence = np.random.permutation(len(data))
    return data[interleaver_sequence]

# De-interleaver
def deinterleave(data):
    np.random.seed(42)
    interleaver_sequence = np.random.permutation(len(data))
    deinterleaver_sequence = np.argsort(interleaver_sequence)
    return data[deinterleaver_sequence]

# BCJR Algorithm (simplified for demonstration)
def bcjr_decoder(received, g, iterations=5):
    n = len(received) // len(g)
    state_prob = np.ones((2**len(g[0]), n+1)) * 0.5
    for _ in range(iterations):
        for t in range(n):
            for state in range(2**len(g[0])):
                for bit in [0, 1]:
                    next_state = (state >> 1) | (bit << (len(g[0])-1))
                    encoded_bits = [(bit + np.dot([int(b) for b in bin(state)[2:].zfill(len(g[0]))], g_i)) % 2 for g_i in g]
                    received_bits = received[t*len(g):(t+1)*len(g)]
                    likelihood = np.prod([1 if e == r else 0.5 for e, r in zip(encoded_bits, received_bits)])
                    state_prob[next_state, t+1] *= likelihood * state_prob[state, t]
        state_prob /= np.sum(state_prob, axis=0)  # Normalize
    decoded = np.argmax(state_prob, axis=0) % 2
    return decoded.astype(int)

# Turbo Encoder
def turbo_encode(data):
    g1 = [np.array([1, 0, 1]), np.array([1, 1, 1])]  # Generator polynomials for encoder 1
    g2 = [np.array([1, 0, 1]), np.array([1, 1, 1])]  # Generator polynomials for encoder 2

    encoded_1 = conv_encoder(data, g1)
    interleaved_data = interleave(data)
    encoded_2 = conv_encoder(interleaved_data, g2)

    return np.concatenate([encoded_1, encoded_2])

# Turbo Decoder (simplified iterative BCJR decoder)
def turbo_decode(received, iterations=5):
    n = len(received) // 2
    received_1 = received[:n]
    received_2 = received[n:]

    decoded_1 = bcjr_decoder(received_1, [np.array([1, 0, 1]), np.array([1, 1, 1])], iterations)
    interleaved_data = interleave(decoded_1)
    decoded_2 = bcjr_decoder(received_2, [np.array([1, 0, 1]), np.array([1, 1, 1])], iterations)
    deinterleaved_data = deinterleave(decoded_2)
    
    return deinterleaved_data

# Function to introduce errors in the encoded message
def introduce_errors(encoded_message, num_errors):
    corrupted_message = encoded_message.copy()
    for _ in range(num_errors):
        error_position = np.random.randint(0, len(corrupted_message))
        corrupted_message[error_position] ^= 1  # Flip the bit to simulate error
    return corrupted_message

# Simulate the original binary data
original_bin_data = np.random.randint(0, 2, 500)  # Simulate a 256-bit binary file

# Encode the binary message
encoded_message = turbo_encode(original_bin_data)

# Introduce errors (simulate data corruption)
num_errors = 5  # Number of bit errors to introduce
corrupted_message = introduce_errors(encoded_message, num_errors)

# Decode the corrupted message
decoded_message = turbo_decode(corrupted_message)

# Check if the decoded message matches the original message
decoding_success = np.array_equal(decoded_message, original_bin_data)
print("Decoding successful:", decoding_success)
if decoding_success:
    print("Decoded message matches original.")
else:
    print("Decoded message does not match original.")


C:\Users\Hendy Group\AppData\Local\Temp\ipykernel_10256\982987294.py:39: RuntimeWarning: invalid value encountered in divide
  state_prob /= np.sum(state_prob, axis=0)  # Normalize


Decoding successful: False
Decoded message does not match original.
